In [1]:
r"""
Use the prepared cutout of ERA5 model level data to prepare capacity factors for AWE, both onshore and
offshore.
"""
import atlite
import functools
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
from operator import itemgetter

In [2]:

def convert_wind_awe(ds, turbine):
    """
    Convert wind speeds for turbine to wind energy generation.
    """
    V, POW, hub_height, P = itemgetter("V", "POW", "hub_height", "P")(turbine)

    #wnd_hub = windm.extrapolate_wind_speed(ds, to_height=hub_height)
    wnd_hub = ds.wind_speed

    def _interpolate(da):
        return np.interp(da, V, POW / P)

    da = xr.apply_ufunc(
        _interpolate,
        wnd_hub,
        input_core_dims=[[]],
        output_core_dims=[[]],
        output_dtypes=[wnd_hub.dtype],
        dask="parallelized",
    )

    da.attrs["units"] = "MWh/MWp"
    da = da.rename("specific generation")

    return da



In [3]:


# TODO https://atlite.readthedocs.io/en/latest/examples/landuse-availability.html

boundaries = "../build/shapes/eez.geojson"
natura2000 = "../data/potentials_offshore/natura2000_areas/eea_v_3035_100_k_natura2000_p_2021_v12_r01/SHP/Natura2000_end2021_rev1_epsg3035.shp"
shipdensity = "../data/potentials_offshore/shipping_density_global/shipdensity_global.tif"
gebco = "../data/potentials_offshore/gebco_2023_sub_ice_topo/GEBCO_2023_sub_ice_topo.nc"
path_cutout = "../build/cutouts/cutout-era5-model-level_adapted_copy.nc"
path_powercurve = "../data/power_curves/AWE_500kw_softwing.csv"
snakemake_output = "../build/capacity_factors/capacity_factors_awe.nc"

df_boundaries = gpd.read_file(boundaries)
cutout = atlite.Cutout(path_cutout)


In [4]:

# prepare power curve
awe_500kw_sw = pd.read_csv(path_powercurve, index_col=0)
awe_500kw_sw = dict(V=np.array(list(awe_500kw_sw.index)), POW=np.array(list(awe_500kw_sw["power"])), hub_height=100)
awe_500kw_sw["P"] = np.max(awe_500kw_sw["POW"])


In [ ]:

# prepare excluder
excluder = atlite.ExclusionContainer()

excluder.add_geometry(natura2000)

max_depth = 60
func = functools.partial(np.greater, -max_depth)
excluder.add_raster(gebco, crs=4326, codes=func)

# prepare availability matrix
availability = cutout.availabilitymatrix(df_boundaries, excluder)

availability.to_netcdf("../build/capacity_factors/availability_awe.nc")


In [6]:
# prepare capacity matrix
# No need to define area per grid cell and capacity per area as we ask for capacity factors per unit in the end
capacity_matrix = availability.rename({"dim_0": "name"}).stack(spatial=["y", "x"])
print(capacity_matrix)

# prepare capacity factors
capfac_wind_awe = cutout.convert_and_aggregate(convert_func=convert_wind_awe, turbine=awe_500kw_sw, per_unit=True, matrix=capacity_matrix)

capfac_wind_awe.to_netcdf(snakemake_output)


<xarray.DataArray (name: 31, spatial: 34937)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * name     (name) int64 0 1 2 3 4 5 6 7 8 9 ... 21 22 23 24 25 26 27 28 29 30
  * spatial  (spatial) object MultiIndex
  * y        (spatial) float64 72.0 72.0 72.0 72.0 72.0 ... 32.0 32.0 32.0 32.0
  * x        (spatial) float64 -17.0 -16.75 -16.5 -16.25 ... 36.5 36.75 37.0
da
<xarray.DataArray 'specific generation' (time: 52584, y: 161, x: 217)>
dask.array<transpose, shape=(52584, 161, 217), dtype=float32, chunksize=(100, 161, 217), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 -17.0 -16.75 -16.5 -16.25 ... 36.25 36.5 36.75 37.0
  * y        (y) float64 72.0 71.75 71.5 71.25 71.0 ... 32.75 32.5 32.25 32.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2018-12-31T23:00

AssertionError: 

In [5]:
availability = xr.load_dataarray("../build/capacity_factors/availability_awe.nc")